<a href="https://colab.research.google.com/github/talhavawda/tic-tac-toe-monte-carlo/blob/master/TicTacToe_MonteCarlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tic Tac Toe using Monte Carlo Tree Search

 ## COMP304 (ARTIFICIAL INTELLIGENCE) 2020 ASSIGNMENT 1
 
 Team: A1GroupWC

 Team Members:
 -  Azhar Mohamed&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;(218006491)
 - Yashlin Naidu &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;(216019492)
 - Ricardo Pillay &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;(218009114)
 - Ahmad Jawaad Shah&ensp;&ensp;&ensp;(218029400)
 - Talha Vawda &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;(218023210)

<br>

## Acknowledgements and References



# Introduction and Background
Talk about game variant 

# Imports and constants
This cell contains all the libraries that is needed to run this game. Furthermore, constant initializations are done here and will be used throughout the notebook


In [ ]:
"""
Imports
"""
import numpy as np
import math
 
 
"""
Constants
"""
# 'C' is used in the UCT formula
C = 2
# Dimension of the board
DIMENSIONS = 4
# Number of tokens needed in a row to win
K = 3


if DIMENSIONS > 4 and DIMENSIONS < 7:
  K = 4
elif DIMENSIONS > 6:
  K = 5
else:
  pass
  


# Board Class

In [ ]:
class Board:
 
 
  def __init__(self):
    pass

# Node Class

In [ ]:
class Node:
 
 
  def __init__(self, board: Board, parent=None):
    """
    Constructor for the Node class

    Parameters:
    board (Board): An instance of the board
    parent (Node): The parent node of the current node
    """
    # The number of wins through this node
    self.v = 0
    # The number of times the node has been rolled out
    self.n = 0
    # The parent Node
    self.parent = parent
    # The Game State
    self.board = board
 
 
  def calculate_uct_value(self):
    """
    A method to calculate the UCT value for the node
    
    Returns:
    int: Float
    """
    # Check for 0 division
    if self.n == 0:
      # Return positive infinity
      return np.inf
    else:
      # Return uct value for this node
      return (self.v/self.n) + C * math.sqrt(np.log(self.parent.n)/self.n)